In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\arbas\\Documents\\Projects\\Machine-Learning\\EndToEnd_Deployed\\Brain_Tumor_Classification'

In [5]:
import sys

In [6]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class Data_Ingestion_Config:
    root_dir: Path
    source_URL: str
    local_data_path : Path 
    unzip_dir: Path

In [7]:
from src.brain_tumor_classifier.constant import *
from src.brain_tumor_classifier.utils.common import read_yaml,create_directories

In [ ]:
def create_directories(file_path):
    pass
def read_yaml(file_path):
    pass
class Configuration_Manager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH,schema_path=SCHEMA_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(config_path)
        self.schema = read_yaml(config_path)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)->Data_Ingestion_Config:
        config = self.config.Data_Ingestion
        create_directories([config.root_dir])
        data_ingestion_config = Data_Ingestion_Config(root_dir=config.root_dir,source_URL=config.source_URL,local_data_path=config.local_data_path,unzip_dir=config.unzip_dir)
        return data_ingestion_config
        

In [ ]:
import os
import urllib.request as request
import zipfile
from src.brain_tumor_classifier.utils.common import get_size, logger


class Data_Ingestion:
    def __init__(self,config:Data_Ingestion_Config):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_path):
            filename,header = request.urlretrieve(url=self.config.source_URL,filename=self.config.local_data_path)
            logger.info(f'Successfully downloaded data {filename} with info: {header} at: {self.config.local_data_path}')
        else:
            logger.info(f'Data already exists of size {get_size(Path(self.config.local_data_path))}')
    
    def extract_data(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_path,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
    def initiate_data_ingestion(self):
        self.download_data()
        self.extract_data()

In [ ]:
try:
    config = Configuration_Manager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = Data_Ingestion(data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
except Exception as e:
    raise e